In [1]:
from qwen2_llm import Qwen

llm = Qwen()
# llm.invoke('加班有哪些补偿？')

/mnt/workspace/lonhai/LangChain_Deploy/my_langChain_venv/my_venv_first/lib/python3.11/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


A6


In [2]:
# embedding模型加载
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "../Qwen/Qwen3-Embedding-0.6B"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
# 加载词嵌入模型
hf = HuggingFaceEmbeddings(
    model_name=model_name,  # 模型名称
    model_kwargs=model_kwargs,  # 模型参数
    encode_kwargs=encode_kwargs  # 模型转向量的参数
)
# hf.embed_query('你是')

In [3]:
#######  2. 数据准备：文档读取和切分 #######
# 文档读取
from langchain_community.document_loaders import PyPDFLoader,Docx2txtLoader
loader = Docx2txtLoader('./data/【修订版】XX公司员工手册.docx') # 定义读取方法和文档
# loader = PyPDFLoader('./data/人事面试常见问题.pdf') # 定义读取方法和文档
document = loader.load() # 进行读取文档
# document[0].page_content # 文档内容
# document[0].metadata # 文档来源

In [4]:
# 文档切分
from langchain_text_splitters.character import CharacterTextSplitter,RecursiveCharacterTextSplitter
# 定义切分方法
spliter = RecursiveCharacterTextSplitter(
                      chunk_size=200,    # 文档块大小（最大）
                      chunk_overlap=30) # 重叠块大小
# 文档分块
chunk_documents = spliter.split_documents(document)
# chunk_documents

In [5]:
#######  3. 向量存储和检索 #######
from langchain_community.vectorstores import FAISS,Qdrant
# 向量存储
vectorstore = FAISS.from_documents(
    documents=chunk_documents,  # 待存储的文档对象
    embedding=hf,  # 词嵌入模型
)
# 测试，相似性文档搜索
query = '加班补偿都有哪些？'
docs = vectorstore.similarity_search(query, k=4)
# docs

In [6]:
#######  4. 检索器 #######
#定义检索器
retriever = vectorstore.as_retriever(
    search_type='similarity', # similarity mmr similarity_score_threshold
    search_kwargs={'k':4,} # 其他搜索参数
)
retriever.invoke(query)

[Document(id='3557f69e-1c4b-4bfe-b640-b61fd092523f', metadata={'source': './data/【修订版】XX公司员工手册.docx'}, page_content='为保障员工晚饭时间，对在工作日确需安排加班的，在 18：30 后的时间才开始计算加班时长。\n\n\t\t对正常工作时间以外的打卡记录，如未按照规定申请加班并经审批通过的，均不被认定为加班，按个人自行处置打卡时间处理。\n\n三．加班补偿\n\n\n\n\t\t调休：结合个人意愿及公司业务需要，在加班之后的其他工作时间安排员工休息与加班时间等时长的时间。调休以小时为单位，但最长不超过三天。按流程经审批通过在'),
 Document(id='ae049df2-7f00-4127-b766-8bd61bd7bf8a', metadata={'source': './data/【修订版】XX公司员工手册.docx'}, page_content='人事行政部备案后即可安排调休。（具体请参照调休申请流程）\n\n\n\n\t对安排加班的员工，公司给予餐费补助，补助标准为￥15 元/每餐，在当月工资中发放。后续公司有调整的，按最新标准执行。对周末及法定节假日，每加班半天补一餐。如当天公司已提供餐食的，不再支付餐费补助。\n\n\t人事行政部根据加班申请核实考勤后，定期进行汇总登记，核定调休时间。员工如有异议，应当月向人事行政部提出复查。'),
 Document(id='fc7d3b36-4f52-4b2e-a6e2-31a57cc1a8bc', metadata={'source': './data/【修订版】XX公司员工手册.docx'}, page_content='第三条 加班\n\n\n\n一. 加班原则\n\n\n\n\t健康优先原则：公司倡导合理安排时间高效工作，各人员在安排加班时，应结合自身身体状况，对加班频次、时间长短及与正常上班时间的间隔做出合理安排，保证身体健康。\n\n\n\n二．加班认定\n\n\n\n\t\t公司实行加班审批制，员工因工作原因确需要加班的，按照流程给中心负责人审批通过后才认定为加班。'),
 Document(id='fc6e0cd7-b127-44c5-8b10-8fd26327dd2d', met

In [9]:
#######  5. 构建QA链 #######
# 编写提示模板
template = '''
你是一个企业新人导师，能够通过阅读文档内容，回答用户的问题。
文档内容如下：
{context}
用户的问题是：{question}
'''
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate([
    ("system", template),
    ("user", "{question}")
])

# 构建链
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm=llm,  # 大模型
    retriever=retriever,  # 检索器
    chain_type_kwargs={'prompt':prompt}
)
# 测试
chain.invoke('加班有哪些补偿？')

你是一个企业新人导师，能够通过阅读文档内容，回答用户的问题。
文档内容如下：
为保障员工晚饭时间，对在工作日确需安排加班的，在 18：30 后的时间才开始计算加班时长。

		对正常工作时间以外的打卡记录，如未按照规定申请加班并经审批通过的，均不被认定为加班，按个人自行处置打卡时间处理。

三．加班补偿



		调休：结合个人意愿及公司业务需要，在加班之后的其他工作时间安排员工休息与加班时间等时长的时间。调休以小时为单位，但最长不超过三天。按流程经审批通过在

人事行政部备案后即可安排调休。（具体请参照调休申请流程）



	对安排加班的员工，公司给予餐费补助，补助标准为￥15 元/每餐，在当月工资中发放。后续公司有调整的，按最新标准执行。对周末及法定节假日，每加班半天补一餐。如当天公司已提供餐食的，不再支付餐费补助。

	人事行政部根据加班申请核实考勤后，定期进行汇总登记，核定调休时间。员工如有异议，应当月向人事行政部提出复查。

第三条 加班



一. 加班原则



	健康优先原则：公司倡导合理安排时间高效工作，各人员在安排加班时，应结合自身身体状况，对加班频次、时间长短及与正常上班时间的间隔做出合理安排，保证身体健康。



二．加班认定



		公司实行加班审批制，员工因工作原因确需要加班的，按照流程给中心负责人审批通过后才认定为加班。

部分员工放假的假日，适逢星期六、星期日，则不补假。



三.  年休假：按照《企业职工带薪年休假条例》的规定，年休假天数与员工连续工作时间



相关：累计工作已满 1 年不满 10 年的，年休假 5 天；已满 10 年不满 20 年的，年休假
用户的问题是：加班有哪些补偿？
~~~~~~~~~~~~
加班有哪些补偿？


{'query': '加班有哪些补偿？', 'result': '- 员工如对调休或补偿有异议，可向人事行政部提出复查申请。'}

In [ ]:
import os
from langchain_deepseek import ChatDeepSeek

# 一定要先配置环境变量
os.environ["DEEPSEEK_API_KEY"] = "sk-ae5ea9125bfe48a680c5dc23caeac538"

# 初始化模型
llm = ChatDeepSeek(
    model="deepseek-chat",  # 或者 "deepseek-reasoner" 根据需求选择模型
    temperature=0.7,
    max_tokens=512,
    timeout=30,
    max_retries=3,
)

template = '''
你是一个企业新人导师，能够通过阅读文档内容，回答用户的问题。
文档内容如下：
{context}
用户的问题是：{question}
'''
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate([
    ("system", template),
    ("user", "{question}")
])

# 构建链
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm=llm,  # 大模型
    retriever=retriever,  # 检索器
    chain_type_kwargs={'prompt':prompt}
)
# 测试
chain.invoke('加班有哪些补偿？')

In [7]:
llm = Qwen('../Qwen/Qwen2.5-0.5B-Instruct')

template = '''
你是一个企业新人导师，能够通过阅读文档内容，回答用户的问题。
文档内容如下：
{context}
用户的问题是：{question}
'''
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate([
    ("system", template),
    ("user", "{question}")
])

# 构建链
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm=llm,  # 大模型
    retriever=retriever,  # 检索器
    chain_type_kwargs={'prompt':prompt}
)
# 测试
chain.invoke('加班有哪些补偿？')

A6
你是一个企业新人导师，能够通过阅读文档内容，回答用户的问题。
文档内容如下：
为保障员工晚饭时间，对在工作日确需安排加班的，在 18：30 后的时间才开始计算加班时长。

		对正常工作时间以外的打卡记录，如未按照规定申请加班并经审批通过的，均不被认定为加班，按个人自行处置打卡时间处理。

三．加班补偿



		调休：结合个人意愿及公司业务需要，在加班之后的其他工作时间安排员工休息与加班时间等时长的时间。调休以小时为单位，但最长不超过三天。按流程经审批通过在

人事行政部备案后即可安排调休。（具体请参照调休申请流程）



	对安排加班的员工，公司给予餐费补助，补助标准为￥15 元/每餐，在当月工资中发放。后续公司有调整的，按最新标准执行。对周末及法定节假日，每加班半天补一餐。如当天公司已提供餐食的，不再支付餐费补助。

	人事行政部根据加班申请核实考勤后，定期进行汇总登记，核定调休时间。员工如有异议，应当月向人事行政部提出复查。

第三条 加班



一. 加班原则



	健康优先原则：公司倡导合理安排时间高效工作，各人员在安排加班时，应结合自身身体状况，对加班频次、时间长短及与正常上班时间的间隔做出合理安排，保证身体健康。



二．加班认定



		公司实行加班审批制，员工因工作原因确需要加班的，按照流程给中心负责人审批通过后才认定为加班。

部分员工放假的假日，适逢星期六、星期日，则不补假。



三.  年休假：按照《企业职工带薪年休假条例》的规定，年休假天数与员工连续工作时间



相关：累计工作已满 1 年不满 10 年的，年休假 5 天；已满 10 年不满 20 年的，年休假
用户的问题是：加班有哪些补偿？
~~~~~~~~~~~~
加班有哪些补偿？


{'query': '加班有哪些补偿？',
 'result': '加班补偿包括餐费补助和调休，具体包括：餐费补助标准为￥15元/每餐，在当月工资中发放。调休时间为小时，最长不超过三天，并需经过审批程序。'}

In [11]:
#######  6. 界面开发 #######
import random
import gradio as gr

# 生成结果的函数，调用链回复
def get_response(message, history):
    return chain.invoke(message)['result']

demo = gr.ChatInterface(
    fn=get_response, # 执行的函数
    chatbot=gr.Chatbot(height=500), # 聊天窗口
    textbox=gr.Textbox(placeholder="请输入问题", container=False, scale=7), # 文字输入
    title="人事面试百事通",
    description="我是一个精通人事面试的助手，你可以问我相关的面试的问题。",
    theme="soft", # 主题色
    submit_btn = "发送",
    stop_btn = "停止",
    )

if __name__ == "__main__":
    demo.launch(server_port=8001,  # 端口
                server_name='127.0.0.1', # 地址
                share=True # 网络穿透
                )

/tmp/ipykernel_50916/2179391905.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500), # 聊天窗口
/mnt/workspace/lonhai/LangChain_Deploy/my_langChain_venv/my_venv_first/lib/python3.11/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:8001

Could not create share link. Missing file: /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.3
3. Move the file to this location: /root/.cache/huggingface/gradio/frpc
